In [1]:
from google.colab import files
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.stattools import jarque_bera
from statsmodels.graphics.api import plot_leverage_resid2, influence_plot
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from itertools import combinations

In [2]:
!pip install snowflake-snowpark-python

In [3]:
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import col, year, month, count, when, quarter
from snowflake.snowpark.functions import  concat, to_date
from snowflake.snowpark.functions import avg, stddev_pop, corr
import numpy as np
import pandas as pd
from snowflake.snowpark.exceptions import SnowparkSQLException
from snowflake.snowpark.functions import col, year, month, count, when, concat_ws, to_date, lpad
from google.colab import files

In [4]:
uploaded = files.upload()

Saving deflator_yearly_agg.csv to deflator_yearly_agg (2).csv
Saving gdp_norm_monthly_agg.csv to gdp_norm_monthly_agg (2).csv
Saving general_cpi_monthly_agg.csv to general_cpi_monthly_agg (2).csv
Saving highway_interurban_yearly_agg.csv to highway_interurban_yearly_agg (2).csv
Saving highway_yearly_agg.csv to highway_yearly_agg (2).csv
Saving Household_debt_yearly_agg.csv to Household_debt_yearly_agg (2).csv
Saving Santiago_gdp_yearly_agg.csv to Santiago_gdp_yearly_agg (2).csv
Saving TCI_monthly_agg.csv to TCI_monthly_agg (2).csv
Saving unemployment_yearly_agg.csv to unemployment_yearly_agg (2).csv


In [5]:
df_deflator = pd.read_csv('deflator_yearly_agg.csv')
df_gdp_norm_monthly = pd.read_csv('gdp_norm_monthly_agg.csv')
df_cpi_monthly = pd.read_csv('general_cpi_monthly_agg.csv')
df_highway_inter = pd.read_csv('highway_interurban_yearly_agg.csv')
df_highway_urban = pd.read_csv('highway_yearly_agg.csv')
df_debt = pd.read_csv('Household_debt_yearly_agg.csv')
df_gdp_santiago = pd.read_csv('Santiago_gdp_yearly_agg.csv')
df_tci_monthly = pd.read_csv('TCI_monthly_agg.csv')
df_unemployment = pd.read_csv('unemployment_yearly_agg.csv')

In [6]:
parameters = {
...    "account": "YEZEPEO-DATASCIENCE",
...    "user": "VNSHARED",
...    "password": "Columbia2024!",
...    "role": "SYSADMIN", # optional
...    "warehouse": "ANALYSIS", # optional
...   "database": "MEIXI_DATA", # optional
...   "schema":"PUBLIC",  # optional
...  }
session= Session.builder.configs(parameters).create()

In [23]:
df_yearly=session.sql('select * from YEARLY_DEFAULT ')
df_yearly=df_yearly.to_pandas()
df_yearly

,YEAR,LIKELIHOODOFDEFAULT
0,2014,0.074191
1,2010,0.094427
2,2011,0.101237
3,2012,0.107576
4,2013,0.068015
5,2018,0.096684
6,2019,0.108227
7,2020,0.145762
8,2023,0.265869
9,2024,0.754162


In [24]:
df_highway_inter.rename(columns={
    'INDEX': 'INDEX_INTERURBAN'}, inplace=True)
df_highway_urban.rename(columns={
     'INDEX': 'INDEX_PLAZA'}, inplace=True)
df_tci_monthly.rename(columns={
    'INDEX': 'INDEX_TCI'}, inplace=True)

Merging ALL df's

In [25]:
# Function to drop duplicate columns from a dataframe that are already in df_yearly
def drop_duplicate_columns(df_to_join, df_master):
    duplicate_columns = [col for col in df_to_join.columns if col in df_master.columns and col != 'YEAR']
    df_to_join = df_to_join.drop(columns=duplicate_columns)
    return df_to_join

# List of dataframes to join with df_yearly
dfs = [
    df_deflator, df_gdp_norm_monthly, df_cpi_monthly, df_highway_inter,
    df_highway_urban, df_debt, df_gdp_santiago, df_tci_monthly, df_unemployment
]


# Perform inner joins on the "YEAR" column
for df in dfs:
    df = drop_duplicate_columns(df, df_yearly)
    df_yearly = df_yearly.merge(df, on='YEAR', how='inner')

df_yearly

,YEAR,LIKELIHOODOFDEFAULT,EXTRACTIONDATE,Country Name,DATE,deflator(annual%),OBSERVATION_DATE,GDP_NORMALIZED,MONTH,REG,...,Date (varchar),Date (#),IPC,INDEX_INTERURBAN,COUNTRY,Debt(%GDP),GLOSA,INDEX_TCI,Monthly_variation(%),Rate(percent)
0,2019,0.108227,2024-03-23,Chile,2019-12-31,2.572598,2019-01-01,100.841334,1,1.0,...,Jan.2019,2019-01-01,76.33,2.167174e+07,Chile,48.068032,ICT Sin Diésel,103.41,1.9,7.2
1,2019,0.108227,2024-03-23,Chile,2019-12-31,2.572598,2019-01-01,100.841334,1,1.0,...,Jan.2019,2019-01-01,76.33,2.167174e+07,Chile,48.068032,ICT Sin Diésel,101.26,-2.1,7.2
2,2019,0.108227,2024-03-23,Chile,2019-12-31,2.572598,2019-01-01,100.841334,1,1.0,...,Jan.2019,2019-01-01,76.33,2.167174e+07,Chile,48.068032,ICT Sin Diésel,99.60,-1.6,7.2
3,2019,0.108227,2024-03-23,Chile,2019-12-31,2.572598,2019-01-01,100.841334,1,1.0,...,Jan.2019,2019-01-01,76.33,2.167174e+07,Chile,48.068032,ICT Sin Diésel,99.95,0.4,7.2
4,2019,0.108227,2024-03-23,Chile,2019-12-31,2.572598,2019-01-01,100.841334,1,1.0,...,Jan.2019,2019-01-01,76.33,2.167174e+07,Chile,48.068032,ICT Sin Diésel,102.00,2.0,7.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,2021,0.153499,2024-03-23,Chile,2021-12-31,6.797841,2021-11-01,102.432177,11,1.0,...,Dec.2021,2021-12-01,86.67,2.206913e+07,Chile,44.767418,ICT Sin Diésel,105.97,1.3,8.9
5036,2021,0.153499,2024-03-23,Chile,2021-12-31,6.797841,2021-11-01,102.432177,11,1.0,...,Dec.2021,2021-12-01,86.67,2.206913e+07,Chile,44.767418,ICT Sin Diésel,106.43,0.4,8.9
5037,2021,0.153499,2024-03-23,Chile,2021-12-31,6.797841,2021-11-01,102.432177,11,1.0,...,Dec.2021,2021-12-01,86.67,2.206913e+07,Chile,44.767418,ICT Sin Diésel,110.18,3.5,8.9
5038,2021,0.153499,2024-03-23,Chile,2021-12-31,6.797841,2021-11-01,102.432177,11,1.0,...,Dec.2021,2021-12-01,86.67,2.206913e+07,Chile,44.767418,ICT Sin Diésel,110.24,0.1,8.9


In [26]:
df_yearly.columns

Index(['YEAR', 'LIKELIHOODOFDEFAULT', 'EXTRACTIONDATE', 'Country Name', 'DATE',
       'deflator(annual%)', 'OBSERVATION_DATE', 'GDP_NORMALIZED', 'MONTH',
       'REG', 'Descripción series', 'Date (varchar)', 'Date (#)', 'IPC',
       'INDEX_INTERURBAN', 'COUNTRY', 'Debt(%GDP)', 'GLOSA', 'INDEX_TCI',
       'Monthly_variation(%)', 'Rate(percent)'],
      dtype='object')

In [27]:
columns_to_remove=['EXTRACTIONDATE', 'Country Name', 'DATE','OBSERVATION_DATE','MONTH',
       'REG', 'Descripción series', 'Date (varchar)', 'Date (#)','COUNTRY','GLOSA','Monthly_variation(%)']
df_yearly = df_yearly.drop(columns=columns_to_remove)

In [28]:
df_yearly.columns

Index(['YEAR', 'LIKELIHOODOFDEFAULT', 'deflator(annual%)', 'GDP_NORMALIZED',
       'IPC', 'INDEX_INTERURBAN', 'Debt(%GDP)', 'INDEX_TCI', 'Rate(percent)'],
      dtype='object')

In [29]:
rename_dict = {
    'deflator(annual%)': 'GDP_DEFLATOR',
    'IPC': 'CPI',
    'GDP(million_pesos)':'SANTIAGO_GDP_MPESOS',
    'Rate(percent)':'UNEMPLOYENT_RATE'
}
df_yearly = df_yearly.rename(columns=rename_dict)

# Display the updated DataFrame
df_yearly


,YEAR,LIKELIHOODOFDEFAULT,GDP_DEFLATOR,GDP_NORMALIZED,CPI,INDEX_INTERURBAN,Debt(%GDP),INDEX_TCI,UNEMPLOYENT_RATE
0,2019,0.108227,2.572598,100.841334,76.33,2.167174e+07,48.068032,103.41,7.2
1,2019,0.108227,2.572598,100.841334,76.33,2.167174e+07,48.068032,101.26,7.2
2,2019,0.108227,2.572598,100.841334,76.33,2.167174e+07,48.068032,99.60,7.2
3,2019,0.108227,2.572598,100.841334,76.33,2.167174e+07,48.068032,99.95,7.2
4,2019,0.108227,2.572598,100.841334,76.33,2.167174e+07,48.068032,102.00,7.2
...,...,...,...,...,...,...,...,...,...
5035,2021,0.153499,6.797841,102.432177,86.67,2.206913e+07,44.767418,105.97,8.9
5036,2021,0.153499,6.797841,102.432177,86.67,2.206913e+07,44.767418,106.43,8.9
5037,2021,0.153499,6.797841,102.432177,86.67,2.206913e+07,44.767418,110.18,8.9
5038,2021,0.153499,6.797841,102.432177,86.67,2.206913e+07,44.767418,110.24,8.9


In [30]:
df_final = df_yearly.groupby('YEAR').mean()
df_final

,LIKELIHOODOFDEFAULT,GDP_DEFLATOR,GDP_NORMALIZED,CPI,INDEX_INTERURBAN,Debt(%GDP),INDEX_TCI,UNEMPLOYENT_RATE
YEAR,,,,,,,,
2019,0.108227,2.572598,100.238281,77.470833,2.167174e+07,48.068032,102.054167,7.2
2020,0.145762,9.637414,95.278923,79.828333,1.624858e+07,48.228743,102.680833,10.8
2021,0.153499,6.797841,100.305252,83.440000,2.206913e+07,44.767418,106.136667,8.9


Merging significant df's

In [40]:
df_yearly=session.sql('select * from YEARLY_DEFAULT ')
df_yearly=df_yearly.to_pandas()

In [41]:
# Function to drop duplicate columns from a dataframe that are already in df_yearly
def drop_duplicate_columns(df_to_join, df_master):
    duplicate_columns = [col for col in df_to_join.columns if col in df_master.columns and col != 'YEAR']
    df_to_join = df_to_join.drop(columns=duplicate_columns)
    return df_to_join

# List of dataframes to join with df_yearly
dfs = [
    df_gdp_norm_monthly, df_cpi_monthly, df_highway_inter,
    df_debt, df_gdp_santiago, df_tci_monthly,
]


# Perform inner joins on the "YEAR" column
for df in dfs:
    df = drop_duplicate_columns(df, df_yearly)
    df_yearly = df_yearly.merge(df, on='YEAR', how='inner')

In [42]:
columns_to_remove=['EXTRACTIONDATE', 'DATE','OBSERVATION_DATE','MONTH',
       'REG', 'Descripción series', 'Date (varchar)', 'Date (#)','COUNTRY','GLOSA','Monthly_variation(%)']
df_yearly = df_yearly.drop(columns=columns_to_remove)

In [ ]:
rename_dict = {
    'IPC': 'CPI',
    'GDP(million_pesos)':'SANTIAGO_GDP_MPESOS',
    'Rate(percent)':'UNEMPLOYENT_RATE'
}
df_yearly = df_yearly.rename(columns=rename_dict)

# Display the updated DataFrame
df_yearly


In [44]:
df_final = df_yearly.groupby('YEAR').mean()
df_final

,LIKELIHOODOFDEFAULT,GDP_NORMALIZED,CPI,INDEX_INTERURBAN,Debt(%GDP),SANTIAGO_GDP_MPESOS,INDEX_TCI
YEAR,,,,,,,
2019,0.108227,100.238281,77.470833,2.167174e+07,48.068032,82149.481470,102.054167
2020,0.145762,95.278923,79.828333,1.624858e+07,48.228743,79602.544458,102.680833
2021,0.153499,100.305252,83.440000,2.206913e+07,44.767418,91828.491152,106.136667


Merging SELECTED df's

In [45]:
df_yearly=session.sql('select * from YEARLY_DEFAULT ')
df_yearly=df_yearly.to_pandas()

In [46]:
# Function to drop duplicate columns from a dataframe that are already in df_yearly
def drop_duplicate_columns(df_to_join, df_master):
    duplicate_columns = [col for col in df_to_join.columns if col in df_master.columns and col != 'YEAR']
    df_to_join = df_to_join.drop(columns=duplicate_columns)
    return df_to_join

# List of dataframes to join with df_yearly
dfs = [
    df_gdp_norm_monthly, df_cpi_monthly, df_highway_inter,
    df_debt
]


# Perform inner joins on the "YEAR" column
for df in dfs:
    df = drop_duplicate_columns(df, df_yearly)
    df_yearly = df_yearly.merge(df, on='YEAR', how='inner')

In [47]:
df_yearly.columns

Index(['YEAR', 'LIKELIHOODOFDEFAULT', 'EXTRACTIONDATE', 'OBSERVATION_DATE',
       'GDP_NORMALIZED', 'MONTH', 'REG', 'Descripción series',
       'Date (varchar)', 'Date (#)', 'IPC', 'DATE', 'INDEX_INTERURBAN',
       'COUNTRY', 'Debt(%GDP)'],
      dtype='object')

In [49]:
columns_to_remove=['EXTRACTIONDATE','OBSERVATION_DATE','MONTH',
       'REG', 'Descripción series', 'Date (varchar)', 'Date (#)','DATE','COUNTRY']
df_yearly = df_yearly.drop(columns=columns_to_remove)

In [50]:
rename_dict = {
    'IPC': 'CPI',
    'GDP(million_pesos)':'SANTIAGO_GDP_MPESOS',
    'Rate(percent)':'UNEMPLOYENT_RATE'
}
df_yearly = df_yearly.rename(columns=rename_dict)

# Display the updated DataFrame
df_yearly


,YEAR,LIKELIHOODOFDEFAULT,GDP_NORMALIZED,CPI,INDEX_INTERURBAN,Debt(%GDP)
0,2014,0.074191,100.770651,65.18,9.557303e+06,39.367669
1,2014,0.074191,100.770651,65.49,9.557303e+06,39.367669
2,2014,0.074191,100.770651,66.04,9.557303e+06,39.367669
3,2014,0.074191,100.770651,66.45,9.557303e+06,39.367669
4,2014,0.074191,100.770651,66.68,9.557303e+06,39.367669
...,...,...,...,...,...,...
1135,2021,0.153499,102.432177,83.45,2.206913e+07,44.767418
1136,2021,0.153499,102.432177,84.44,2.206913e+07,44.767418
1137,2021,0.153499,102.432177,85.57,2.206913e+07,44.767418
1138,2021,0.153499,102.432177,85.99,2.206913e+07,44.767418


In [51]:
df_final = df_yearly.groupby('YEAR').mean()
df_final

,LIKELIHOODOFDEFAULT,GDP_NORMALIZED,CPI,INDEX_INTERURBAN,Debt(%GDP)
YEAR,,,,,
2014,0.074191,100.393099,66.911667,9.557303e+06,39.367669
2015,0.081544,100.187323,69.823333,1.048325e+07,41.261339
2016,0.099478,99.788597,72.466667,1.182673e+07,42.430255
2017,0.099008,99.671862,74.048333,1.352149e+07,43.798892
2018,0.096684,100.802870,75.761667,1.735575e+07,45.836085
2019,0.108227,100.238281,77.470833,2.167174e+07,48.068032
2020,0.145762,95.278923,79.828333,1.624858e+07,48.228743
2021,0.153499,100.305252,83.440000,2.206913e+07,44.767418
